# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

### Imports

In [42]:
import pandas as pd
import numpy as np
from mpmath import *

## Leitura do excel

In [43]:
leitura = pd.ExcelFile('tweets_Nike_1.xlsx')
tweets_treinamento = pd.read_excel(leitura, 'Treinamento')
tweets_teste = pd.read_excel(leitura, 'Teste') 

## Limpando os tweets

In [44]:
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace(':', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('@', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('*', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('“', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('.', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace(')', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('(', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('_', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace(';', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace(',', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('&', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('/', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('#', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('%', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('$', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("!", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("?", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("-", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("+", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("=", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('rt', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("\n", '', regex=False)

In [45]:
tweets_teste.Teste = tweets_teste.Teste.str.replace(':', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('@', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('*', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('“', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('.', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace(')', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('(', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('_', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace(';', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace(',', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('&', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('/', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('#', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('%', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('$', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("!", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("?", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("-", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("+", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("=", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('rt', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("\n", '')

## Adicionando os tweets limpos em uma lista

Nesta célula, criamos uma lista na qual os tweets da tabela treinamento e teste serão adicionados a uma tabela. Os tweets foram separados nas tabelas em relevantes e irrelevantes.

Tweets relevantes: são tweets que interferem diretamente na marca, ou seja, podem interferir na quantidade de clientes para a marca. 
    Tweets irrelevantes: sãao tweets que nao interferem na marca, ou seja, memes, piadas ou algo como "usei nike ontem" ARRUAMAR!!!!

In [46]:
rel_treinamento = tweets_treinamento[(tweets_treinamento['classificacao'] == 1)]
irrel_treinamento = tweets_treinamento[(tweets_treinamento['classificacao'] == 0)]
tweets_treinamento['Treinamento'] = tweets_treinamento

rel_teste = tweets_teste[(tweets_teste['classificacao'] == 1)]
irrel_teste = tweets_teste[(tweets_teste['classificacao'] == 0)]
tweets_teste['Teste'] = tweets_teste

## Split e tirando capslock

Nesta célula criamos novas listas vazias, nas quais vão receber as strings de cada tweet separados e todos sem capslock, caso exista.

In [47]:
lrel= []
lirrel= []
ltodos = []

for tweet in tweets_treinamento['Treinamento']:
    tweet = tweet.lower()
    
for tweet in rel_treinamento['Treinamento']:
    lrel.append(tweet.split())
    
for tweet in irrel_treinamento['Treinamento']:
    lirrel.append(tweet.split())
    
for tweet in tweets_treinamento['Treinamento']:
    ltodos.append(tweet.split())  

## Contando número de tweets e calculando a probabilidade

In [48]:
#--------------------------- contando quantidade de cada tweet marcado como relevante e irrelevante ---------------------------#
qnt_irrel = tweets_treinamento.classificacao[tweets_treinamento.classificacao == 0].count()
qnt_rel = tweets_treinamento.classificacao[tweets_treinamento.classificacao == 1].count()
total_qnt = qnt_rel + qnt_irrel

#------------------------------------ calculando a P de um tweet ser relevante ou irrelevante ---------------------------------#
prob_irrelevante = qnt_irrel/ total_qnt
prob_relevante = qnt_rel/ total_qnt

#------------------------------------- criando dicionarios vazios para a contagem de palavras ---------------------------------#
dic_rel = {}
dic_irrel = {}
dic_tweet = {}

#--------------------------------------- criando listas vazias para as palavras que aparecem ---------------------------------# 
Ln_rel = []
Ln_irrel = []
TD = []
#------------------------------------- contando quantas vezes cada palavra aparece nos tweets --------------------------------#
for frase in lrel:
    for palavrinha in frase:
        Ln_rel.append(palavrinha)
        dic_rel[palavrinha] = Ln_rel.count(palavrinha)

for frase in lirrel:
    for palavrinha in frase:
        Ln_irrel.append(palavrinha)
        dic_irrel[palavrinha] = Ln_irrel.count(palavrinha)

for frase in ltodos:
    for palavrinha in frase:
        TD.append(palavrinha)
        dic_tweet[palavrinha] = TD.count(palavrinha)

## Descobrindo os tamanhos das listas e adicionando todos os tweets separados a uma lista 

In [49]:
print(len(TD))
print(len(Ln_rel))
print(len(Ln_irrel))

#--------------------------------------------- criando uma lista que tera as palavras sem repetição --------------------------#
sem_repeticao= list(set(TD))
print(len(sem_repeticao))
print(len(TD))

5380
378
5002
1593
5380


## Teorema de Bayers 

In [50]:
#------------ criando dicionarios para descobrir a probabilidade da palavra ser relevante ou irrelevante ---------------------#
relevantes = {}
irrelevantes = {}

for palavra in Ln_rel:
    relevantes[palavra] = (dic_rel[palavra] + 1)/ (len(sem_repeticao) + len(Ln_rel))

for palavra in Ln_irrel:
    relevantes[palavra] = (dic_irrel[palavra] + 1)/ (len(sem_repeticao) + len(Ln_irrel))

In [51]:
r = len(Ln_rel)/len(TD)
ir = len(Ln_irrel)/len(TD)

soma_rel = sum(relevantes.values())
soma_irrel = sum(irrelevantes.values())

print(soma_rel + soma_irrel)

1.0336602495086897


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [52]:
classific = []
for tweet in tweets_teste['Teste']:
    tweets_frase = []
    tweets_frase.append(tweet.split())
    for palavra in tweets_frase:
        for palavrinha in palavra:
            tweets_frase = []
            tweets_frase.append(palavrinha)                          
            for palavra in tweets_frase:
                Probfrase_rel = 1
                Probtweets_rel = 0
                Probfrase_irrel = 1
                Probtweets_irrel = 0
                
                if palavra in relevantes:
                    Probtweets_r = relevantes[palavra]
                    Probfrase_rel *= Probtweets_r
                elif palavra in irrelevantes:
                    Probtweets_ir = irrelevantes[palavra]
                    Probfrase_rel *= Probtweets_ir
                else:
                    Probtweets_ir = 1/(len(Ln_irrel) + len(sem_repeticao))
                    Probtweets_rel = 1/(len(Ln_rel) + len(sem_repeticao))
                    Probfrase_rel *= Probtweets_rel
                    Probfrase_irrel *= Probtweets_irrel
    if Probfrase_rel > Probfrase_irrel:
        classific.append(1)
    else:
        classific.append(0)
        
tweets_teste['Computador'] = classific

verdadeiro_positivo = 0
verdadeiro_negativo = 0
falso_positivo = 0
falso_negativo = 0

for i in range(len(tweets_teste)):
    if tweets_teste['Computador'][i] == 1 and tweets_teste['classificacao'][i] == 1:
        verdadeiro_positivo += 1
    elif tweets_teste['Computador'][i] == 0 and tweets_teste['classificacao'][i] == 1:
        falso_negativo += 1
    elif tweets_teste['Computador'][i] == 0 and tweets_teste['classificacao'][i] == 0:
        verdadeiro_negativo += 1
    else:
        falso_positivo += 1

In [53]:
print('A porcentagem de positivos verdadeiros é:', verdadeiro_positivo/2,'%')
print('A porcentagem de negativos verdadeiros é:', verdadeiro_negativo/2,'%')
print('A porcentagem de positivos falsos é:', falso_positivo/2,'%')
print('A porcentagem de negativos falsos é:', falso_negativo/2,'%')

A porcentagem de positivos verdadeiros é: 4.5 %
A porcentagem de negativos verdadeiros é: 47.0 %
A porcentagem de positivos falsos é: 41.5 %
A porcentagem de negativos falsos é: 7.0 %


In [ ]:
print('Quantidade dos respectivos Positivos e Negativos (sem porcentagem):')
tweets_treinamento.Novas_Medidas.value_counts()

In [ ]:
import matplotlib.pyplot as plt
r = df2.Novas_Medidas.value_counts()/len(Res)*100
t = r.reindex(['positivos falsos','positivos verdadeiros','negativos verdadeiros','negativos falsos'])
plot = t.plot(kind='pie',title='Novas Medidas exigidas pelo professor (%)',autopct='%.1f',figsize=(7, 7))
plt.show()

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


Pode-se concluir que o programa não é eficaz. O valor alto dos positivos falsos, se dá pela citação do nome da marca nos tweets de forma que não seja relevante a empresa, como um meme ou uma piada. A porcentagem de negativos falsos pode ser explicada pelo padrão que os tweets irrelevantes seguem, facilitando a análise. O valor de negativos falsos ser baixo, significa uma coisa boa pois, não receber uma informação que seja relevante para a sua empresa significa deixar de receber um feedback de um usuário. Apesar de receber informações desnecessárias, possuir um percentual alto de positivos falso é melhor do que um percentual alto de negativo falso, porque assim, a empresa perdeu pouca informação relevante. O valor de positivo verdadeiro ser baixo, significa que poucas palavras nos tweets são positivas, e para um melhor análise, teríamos que ter uma base de dados bem maior.

O programa não é preciso pois, existem fatores que comprometem a análise dos tweets, como o sarcasmo. Não podemos confundi-lo com ironia, já que o sarcasmo é a forma de que as pessoas utilizam geralmente para criticar algo. O problema do sarcasmo é que ele é identificado através da pontuação da frase e para que isso seja possível, precisaria de um programa que analisasse tais pontuações, sendo que provavelmente definiria como positivos algumas dessas frases, e mesmo assim não seria totalmente preciso. Já os casos de dupla negação, são aquele que o usuário geralmente escreve a negação por acidente. Um exemplo é a frase “não vi ninguém”, onde que a análise mostra que ela viu uma pessoa, porém ela quis dizer o oposto.  O programa então julgaria negativo algo que era positivo ou o contrário.

Se essas e outras alterações fossem aplicadas ao nosso programa, a empresa Nike poderia planejar um processo de expansão do nosso projeto, fazendo com que tivesse uma busca mais ampla, não somente a análise do Twitter, mas em outras grandes redes socias, como Facebook, Instagram, dentre outros. Isso faria com que a empresa tivesse um melhor feedback dos seus usuários, podendo então, desenvolver soluções mais eficazes para uma melhor satisfação.